In [1]:
import gc
import pandas as pd
import numpy as np
from sklearn import metrics#, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings
import itertools

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
#path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

In [2]:
#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb02A = pd.read_csv(path+'../sub/lgCV_0206A.csv.gz', compression='gzip')
lgb09 = pd.read_csv(path+'../sub/lgCV_0906.csv.gz', compression='gzip')
lgb10 = pd.read_csv(path+'../sub/lgCV_1006.csv.gz', compression='gzip')
lgb11A= pd.read_csv(path+'../sub/lgCV_1106A.csv.gz', compression='gzip')
lgb11D= pd.read_csv(path+'../sub/lgCV_1106D.csv.gz', compression='gzip')
lgb14= pd.read_csv(path+'../sub/lgCV_1406.csv.gz', compression='gzip')
lgb14A= pd.read_csv(path+'../sub/lgCV_1406A.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path+'../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path+'../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path+'../sub/lgCV_0206.csv.gz', compression='gzip')
lgb17 = pd.read_csv(path+'../sub/lgCV_1706.csv.gz', compression='gzip')
lgb19 = pd.read_csv(path+'../sub/lgCV_1906.csv.gz', compression='gzip')
lgb19A= pd.read_csv(path+'../sub/lgCV_1906A.csv.gz', compression='gzip')
rnn =   pd.read_csv(path+'../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path+'../sub/rnnCV_2705A.csv.gz', compression='gzip')
rnn12 = pd.read_csv(path+'../sub/rnnCV_1206.csv.gz', compression='gzip')
mlp =   pd.read_csv(path+'../sub/mlpCV_2505.csv.gz', compression='gzip')
rdgv19 =   pd.read_csv(path+'../sub/rdgv19CV_1606.csv.gz', compression='gzip')
truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [3]:
lgb14['deal_probability'] =  ( lgb14['deal_probability'].values + lgb14A['deal_probability'].values)*0.5
lgb17['deal_probability'] =  ( lgb17['deal_probability'].values + lgb19['deal_probability'].values)*0.25


In [4]:
#img_cts = truth['image_top_1'].value_counts().reset_index()
#truth['image_top_big'] = 'lo_count'
#big_top_1 = img_cts[img_cts['image_top_1']>5000]['index'].tolist()
#idx = truth['image_top_1'].isin(big_top_1)
#truth['image_top_big'][idx] = truth['image_top_1'][idx].astype(str).values

def keep_big(df, col, cutoff):
    cts = df[col].value_counts().reset_index()
    df[col+'_big'] = 'lo_count'
    big = cts[cts[col]>cutoff]['index'].tolist()
    idx = df[col].isin(big)
    df[col+'_big'][idx] = df[col][idx].astype(str).values
    return df[col+'_big'].values
truth['image_top_1_big'] = keep_big(truth, 'image_top_1', 5000)
truth['param_1_big'] = keep_big(truth, 'param_1', 5000)
truth['param_2_big'] = keep_big(truth, 'param_2', 5000)
truth['param_3_big'] = keep_big(truth, 'param_3', 5000)
truth['city_big'] = keep_big(truth, 'city', 5000)

In [5]:
truth[[c for c in truth.columns if 'big' in c]].head()

,image_top_1_big,param_1_big,param_2_big,param_3_big,city_big
0,lo_count,Постельные принадлежности,lo_count,lo_count,Екатеринбург
1,lo_count,Другое,lo_count,lo_count,Самара
2,lo_count,lo_count,lo_count,lo_count,Ростов-на-Дону
3,796.0,Автомобильные кресла,lo_count,lo_count,Набережные Челны
4,2264.0,С пробегом,ВАЗ (LADA),lo_count,Волгоград


In [6]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability': 'lgb02_preds' }, inplace=True)
lgb09.rename(columns={'deal_probability': 'lgb09_preds' }, inplace=True)
lgb10.rename(columns={'deal_probability': 'lgb10_preds' }, inplace=True)
lgb11D.rename(columns={'deal_probability': 'lgb11D_preds' }, inplace=True)
lgb11A.rename(columns={'deal_probability': 'lgb11A_preds' }, inplace=True)
lgb14.rename(columns={'deal_probability': 'lgb14_preds' }, inplace=True)
lgb17.rename(columns={'deal_probability': 'lgb17_preds' }, inplace=True)
lgb19A.rename(columns={'deal_probability': 'lgb19A_preds' }, inplace=True)
lgb02A.rename(columns={'deal_probability': 'lgb02A_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
rnn12.rename(columns={'deal_probability': 'rnn12_preds' }, inplace=True)
rdgv19.rename(columns={'deal_probability': 'rdgv19_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(lgb09, on='item_id')\
                .merge(lgb10, on='item_id')\
                .merge(lgb11A, on='item_id')\
                .merge(lgb11D, on='item_id')\
                .merge(lgb14, on='item_id')\
                .merge(lgb17, on='item_id')\
                .merge(lgb19A, on='item_id')\
                .merge(lgb02A, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(rnn12, on='item_id')\
                .merge(rdgv19, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [7]:
pred_cols = [col for col in preds_df.columns if ('_preds' in col) \
             and ('lgb27' not in col) and ('lgb02_' not in col) and ('lgb11A_' not in col) \
             and ('rdgv19_' not in col)]

preds_df['preds_sum'] = preds_df[pred_cols].sum(axis=1)
pred_cols

['rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb09_preds',
 'lgb10_preds',
 'lgb11D_preds',
 'lgb14_preds',
 'lgb17_preds',
 'lgb19A_preds',
 'lgb02A_preds',
 'rnn27_preds',
 'rnn12_preds']

In [8]:
preds_df['price'].fillna(-1,inplace=True)
preds_df['max'] = np.max(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['med'] = np.median(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in pred_cols]),axis=0)

In [9]:
import itertools
for p1, p2 in itertools.combinations(pred_cols, 2):
    print('difference_%s__%s'%(p1,p2))
    preds_df['difference_%s__%s'%(p1,p2)] = preds_df[p2] - preds_df[p1]
    preds_df['sums_%s__%s'%(p1,p2)] = preds_df[p2] + preds_df[p1]

difference_rnn_preds__mlp_preds
difference_rnn_preds__lgb31_preds
difference_rnn_preds__lgb09_preds
difference_rnn_preds__lgb10_preds
difference_rnn_preds__lgb11D_preds
difference_rnn_preds__lgb14_preds
difference_rnn_preds__lgb17_preds
difference_rnn_preds__lgb19A_preds
difference_rnn_preds__lgb02A_preds
difference_rnn_preds__rnn27_preds
difference_rnn_preds__rnn12_preds
difference_mlp_preds__lgb31_preds
difference_mlp_preds__lgb09_preds
difference_mlp_preds__lgb10_preds
difference_mlp_preds__lgb11D_preds
difference_mlp_preds__lgb14_preds
difference_mlp_preds__lgb17_preds
difference_mlp_preds__lgb19A_preds
difference_mlp_preds__lgb02A_preds
difference_mlp_preds__rnn27_preds
difference_mlp_preds__rnn12_preds
difference_lgb31_preds__lgb09_preds
difference_lgb31_preds__lgb10_preds
difference_lgb31_preds__lgb11D_preds
difference_lgb31_preds__lgb14_preds
difference_lgb31_preds__lgb17_preds
difference_lgb31_preds__lgb19A_preds
difference_lgb31_preds__lgb02A_preds
difference_lgb31_preds__rnn

In [10]:
gc.collect()

570

In [11]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb11D_preds,...,difference_lgb19A_preds__rnn27_preds,sums_lgb19A_preds__rnn27_preds,difference_lgb19A_preds__rnn12_preds,sums_lgb19A_preds__rnn12_preds,difference_lgb02A_preds__rnn27_preds,sums_lgb02A_preds__rnn27_preds,difference_lgb02A_preds__rnn12_preds,sums_lgb02A_preds__rnn12_preds,difference_rnn27_preds__rnn12_preds,sums_rnn27_preds__rnn12_preds
0,b912c3c6a6ad,0.091656,0.069891,0.091223,0.083219,0.090855,0.109624,0.107764,0.095101,0.092921,...,-0.017792,0.154454,-0.009246,0.163000,-0.041287,0.177948,-0.032740,0.186494,0.008546,0.145208
1,2dac0150717d,0.147801,0.075968,0.211844,0.157973,0.119609,0.157466,0.161107,0.140269,0.144395,...,-0.067602,0.208879,-0.049681,0.226800,-0.089406,0.230683,-0.071485,0.248604,0.017921,0.159198
2,ba83aefab5dc,0.186261,0.167167,0.236579,0.227048,0.258980,0.245236,0.219107,0.234076,0.222678,...,-0.061566,0.385415,-0.075465,0.371516,-0.079412,0.403260,-0.093311,0.389361,-0.013899,0.309950
3,02996f1dd2ea,0.241699,0.260876,0.383699,0.231447,0.284130,0.289101,0.255519,0.241802,0.271355,...,0.044863,0.512745,0.014382,0.482264,-0.057652,0.615259,-0.088132,0.584779,-0.030481,0.527127
4,7c90be56d2ab,0.407423,0.431520,0.420657,0.412563,0.430581,0.511145,0.511676,0.504849,0.503961,...,-0.154635,0.855203,-0.083575,0.926263,-0.071704,0.772273,-0.000645,0.843332,0.071059,0.771628


### Train scores

In [12]:
pred_cols = [col for col in preds_df.columns if ('preds' in col) 
             and ('difference' not in col) 
             and ('sum' not in col)]
pred_cols

['lgb27_preds',
 'rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb02_preds',
 'lgb09_preds',
 'lgb10_preds',
 'lgb11A_preds',
 'lgb11D_preds',
 'lgb14_preds',
 'lgb17_preds',
 'lgb19A_preds',
 'lgb02A_preds',
 'rnn27_preds',
 'rnn12_preds',
 'rdgv19_preds']

In [13]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if ('_preds' in c) and ('difference' not in c) and ('sum' not in c)]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE lgb02_preds:  0.21625386267187377
RMSE lgb09_preds:  0.21581543265040168
RMSE lgb10_preds:  0.21533498019117664
RMSE lgb11A_preds:  0.21403812331071156
RMSE lgb11D_preds:  0.21392147604708842
RMSE lgb14_preds:  0.21329729304260686
RMSE lgb17_preds:  0.21311496843894262
RMSE lgb19A_preds:  0.21341739569735188
RMSE lgb02A_preds:  0.21627136604335995
RMSE rnn27_preds:  0.21698040871120608
RMSE rnn12_preds:  0.2167114508342396
RMSE rdgv19_preds:  0.24593955235646558


In [14]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [15]:
scores = []
for col in [c for c in preds_df.columns if ('_preds' in c) \
            and ('difference' not in c) \
            and ('sum' not in c)]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
3,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
4,lgb02_preds,0.216371,0.216447,0.217354,0.217032,0.213158
5,lgb09_preds,0.215514,0.215678,0.218418,0.216073,0.212293
6,lgb10_preds,0.215240,0.215533,0.217072,0.215802,0.212023
7,lgb11A_preds,0.215011,0.215205,0.211879,0.215771,0.211735
8,lgb11D_preds,0.214818,0.215074,0.211837,0.215720,0.211549
9,lgb14_preds,0.214204,0.214441,0.211074,0.215146,0.211051


### Correlations in test and train

In [16]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c) \
                                                  and ('difference' not in c) and ('sum' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb11D_preds,lgb14_preds,lgb17_preds,lgb19A_preds,lgb02A_preds,rnn27_preds,rnn12_preds,rdgv19_preds
lgb27_preds,1.000000,0.915952,0.919171,0.990750,0.988682,0.978331,0.978298,0.966819,0.965873,0.961540,0.963754,0.957731,0.983143,0.916804,0.920576,0.568406
rnn_preds,0.915952,1.000000,0.929710,0.910602,0.913307,0.908867,0.913286,0.915655,0.913936,0.912870,0.915690,0.910086,0.911443,0.985360,0.978274,0.551100
mlp_preds,0.919171,0.929710,1.000000,0.914307,0.916343,0.909521,0.913883,0.915367,0.913018,0.911656,0.915252,0.908837,0.913600,0.925010,0.929518,0.577280
lgb31_preds,0.990750,0.910602,0.914307,1.000000,0.991984,0.981891,0.981375,0.966656,0.965897,0.961575,0.963672,0.957796,0.986851,0.914204,0.916939,0.570034
lgb02_preds,0.988682,0.913307,0.916343,0.991984,1.000000,0.982596,0.982052,0.970858,0.969933,0.966325,0.968201,0.962391,0.987434,0.916726,0.920105,0.567698
lgb09_preds,0.978331,0.908867,0.909521,0.981891,0.982596,1.000000,0.990447,0.976755,0.976540,0.972392,0.972977,0.968260,0.988708,0.912375,0.915090,0.560909
lgb10_preds,0.978298,0.913286,0.913883,0.981375,0.982052,0.990447,1.000000,0.982841,0.982033,0.977396,0.978691,0.973449,0.985858,0.916436,0.920034,0.562639
lgb11A_preds,0.966819,0.915655,0.915367,0.966656,0.970858,0.976755,0.982841,1.000000,0.992400,0.988703,0.989962,0.984745,0.971893,0.918727,0.923137,0.558477
lgb11D_preds,0.965873,0.913936,0.913018,0.965897,0.969933,0.976540,0.982033,0.992400,1.000000,0.989978,0.990876,0.985921,0.971846,0.917196,0.921322,0.557072
lgb14_preds,0.961540,0.912870,0.911656,0.961575,0.966325,0.972392,0.977396,0.988703,0.989978,1.000000,0.994906,0.990040,0.967560,0.916160,0.920547,0.553930


In [17]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  \
                                                  and ('difference' not in c) and ('sum' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb11D_preds,lgb14_preds,lgb17_preds,lgb19A_preds,lgb02A_preds,rnn27_preds,rnn12_preds,rdgv19_preds
lgb27_preds,1.000000,0.930288,0.924283,0.991439,0.989502,0.979729,0.979341,0.975840,0.973725,0.970062,0.972503,0.967059,0.984742,0.930692,0.932951,0.571110
rnn_preds,0.930288,1.000000,0.939951,0.927835,0.927867,0.926158,0.928367,0.927119,0.925632,0.924680,0.926976,0.922491,0.928496,0.988198,0.984113,0.559376
mlp_preds,0.924283,0.939951,1.000000,0.922413,0.921927,0.918064,0.920496,0.919275,0.917496,0.916323,0.919199,0.914513,0.922121,0.935069,0.936609,0.561935
lgb31_preds,0.991439,0.927835,0.922413,1.000000,0.993168,0.982874,0.982519,0.978877,0.976777,0.973170,0.975607,0.970094,0.988043,0.931468,0.932981,0.572757
lgb02_preds,0.989502,0.927867,0.921927,0.993168,1.000000,0.984427,0.983525,0.980105,0.978173,0.975031,0.977087,0.971908,0.989577,0.931447,0.933333,0.570342
lgb09_preds,0.979729,0.926158,0.918064,0.982874,0.984427,1.000000,0.991413,0.988267,0.986666,0.983504,0.984455,0.980115,0.989106,0.929748,0.931167,0.565333
lgb10_preds,0.979341,0.928367,0.920496,0.982519,0.983525,0.991413,1.000000,0.992340,0.990473,0.986869,0.988382,0.983483,0.986654,0.931841,0.933902,0.566736
lgb11A_preds,0.975840,0.927119,0.919275,0.978877,0.980105,0.988267,0.992340,1.000000,0.993474,0.990047,0.991264,0.986600,0.983556,0.930518,0.932743,0.563841
lgb11D_preds,0.973725,0.925632,0.917496,0.976777,0.978173,0.986666,0.990473,0.993474,1.000000,0.991525,0.992446,0.988002,0.981933,0.929068,0.931133,0.561824
lgb14_preds,0.970062,0.924680,0.916323,0.973170,0.975031,0.983504,0.986869,0.990047,0.991525,1.000000,0.995582,0.991390,0.978724,0.928224,0.930421,0.558673


In [18]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std', 'med', 'item_seq_number']
categories = ['region', 'param_1_big', 'parent_category_name', 'category_name', \
              'param_2_big', 'param_3_big', 'city_big', 'user_type', 'image_top_1_big']#,
cols += categories
cols = list(set(cols))

In [19]:
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))

In [20]:
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [21]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [22]:
len(train_X)

1202739

In [23]:
%%time
gc.collect()
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.4,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
# [2796]	training's rmse: 0.208275	valid_1's rmse: 0.212167
# [2500]	training's rmse: 0.208429	valid_1's rmse: 0.212098
# [2107]	training's rmse: 0.208771	valid_1's rmse: 0.212027

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.21913	valid_1's rmse: 0.219519
[200]	training's rmse: 0.212703	valid_1's rmse: 0.213329
[300]	training's rmse: 0.211577	valid_1's rmse: 0.212398
[400]	training's rmse: 0.21122	valid_1's rmse: 0.21222
[500]	training's rmse: 0.210991	valid_1's rmse: 0.212155
[600]	training's rmse: 0.210797	valid_1's rmse: 0.212119
[700]	training's rmse: 0.210629	valid_1's rmse: 0.212101
[800]	training's rmse: 0.210476	valid_1's rmse: 0.212088
[900]	training's rmse: 0.210329	valid_1's rmse: 0.212082
[1000]	training's rmse: 0.21019	valid_1's rmse: 0.212073
[1100]	training's rmse: 0.210052	valid_1's rmse: 0.212066
[1200]	training's rmse: 0.209913	valid_1's rmse: 0.21206
[1300]	training's rmse: 0.209782	valid_1's rmse: 0.212054
[1400]	training's rmse: 0.209651	valid_1's rmse: 0.21205
[1500]	training's rmse: 0.209522	valid_1's rmse: 0.212046
[1600]	training's rmse: 0.209395	valid_1's rmse: 0.212045
[1700]	training's rmse: 0

In [24]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

[(4355, 'city_big'),
 (3648, 'param_1_big'),
 (2672, 'category_name'),
 (2254, 'region'),
 (1635, 'image_top_1_big'),
 (1090, 'difference_rnn_preds__rnn27_preds'),
 (1067, 'rdgv19_preds'),
 (867, 'difference_rnn_preds__rnn12_preds'),
 (845, 'param_3_big'),
 (704, 'item_seq_number'),
 (664, 'difference_lgb11D_preds__lgb14_preds'),
 (640, 'difference_lgb11D_preds__lgb17_preds'),
 (607, 'difference_rnn27_preds__rnn12_preds'),
 (602, 'price'),
 (590, 'param_2_big'),
 (558, 'difference_lgb14_preds__lgb19A_preds'),
 (557, 'difference_lgb10_preds__lgb17_preds'),
 (552, 'difference_lgb31_preds__lgb19A_preds'),
 (547, 'difference_lgb31_preds__lgb17_preds'),
 (546, 'difference_lgb09_preds__lgb10_preds'),
 (538, 'difference_mlp_preds__rnn27_preds'),
 (535, 'difference_lgb14_preds__lgb17_preds'),
 (533, 'difference_mlp_preds__rnn12_preds'),
 (530, 'difference_lgb09_preds__lgb02A_preds'),
 (529, 'difference_lgb17_preds__lgb19A_preds'),
 (525, 'sums_mlp_preds__lgb17_preds'),
 (518, 'difference_lgb31

In [28]:
n_estimators = 2500
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [29]:
%%time
y_predls = []
for i in range(5):
    clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.4,
                    num_leaves=32, 
                    seed = i, 
                    learning_rate=.01)#, device='gpu')
    clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
    y_predls.append(clf.predict(test_df[cols]))

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.219217
[200]	training's rmse: 0.212833
[300]	training's rmse: 0.211744
[400]	training's rmse: 0.21142
[500]	training's rmse: 0.211216
[600]	training's rmse: 0.211049
[700]	training's rmse: 0.210905
[800]	training's rmse: 0.210775
[900]	training's rmse: 0.210649
[1000]	training's rmse: 0.210527
[1100]	training's rmse: 0.210413
[1200]	training's rmse: 0.210301
[1300]	training's rmse: 0.210193
[1400]	training's rmse: 0.210086
[1500]	training's rmse: 0.209978
[1600]	training's rmse: 0.209869
[1700]	training's rmse: 0.209764
[1800]	training's rmse: 0.20966
[1900]	training's rmse: 0.209557
[2000]	training's rmse: 0.209457
[2100]	training's rmse: 0.209355
[2200]	training's rmse: 0.209251
[2300]	training's rmse: 0.209149
[2400]	training's rmse: 0.209047
[2500]	training's rmse: 0.208947
Did not meet early stopping. Best iteration is:
[2500]	training's rmse: 0.208947
Training until validation scores don't impr

In [30]:
test_df['deal_probability'] = sum(y_predls)/len(y_predls)
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_2006L2.csv.gz', compression='gzip', index=False, header=True)
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.446934
1503425,65b9484d670f,0.155028
1503426,8bab230b2ecd,0.171616
1503427,8e348601fefc,0.105203
1503428,8bd2fe400b89,0.164077
